In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [ ]:
### stevejobs.txt

In [ ]:
# 불러오기
with open('../data/stevejobs.txt', 'r', encoding='utf8') as f:
    rows=f.readlines()
    lines=[row for row in rows]
text=' '.join(lines)

In [ ]:
# 1-1.텍스트 전처리(정규화): 클렌징
import re
def cleaning(text):
    p = re.compile('[^ ㄱ-ㅣ가-힣]+')#영어 ('[^ a-zA-Z0-9\.]+') ('[^ a-zA-Zㄱ-ㅣ가-힣]+')
result = p.sub('',text).lower()
    return result
df['content']=df['content'].apply(cleaning)
#방법2
import re
df['content'] = df['content'].apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', '', x))

#방법3 영어
from nltk.corpus import stopwords
eng_stop = stopwords.words('english')

In [ ]:
# 1-2.토큰화(문장)
from konlpy.tag import Okt
okt = Okt()
#방법1
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem = True)
    return tokens_ko
#방법2
def okt_tokenizer(text):   # 자르기 + 조사 삭제
    words=okt.pos(text, stem=True)
    filtered_words=[]
    for word, pos in words:
        if pos not in ['Josa']:
            filtered_words.append(word)
    return filtered_words

#방법0
import nltk
nltk.download('punkt')
sentences=nltk.sent_tokenize(text=text)

In [ ]:
# 토큰화(단어) extend: 리스트로 
word_token=[]
for sentence in sentences:
    words=nltk.word_tokenize(sentence)
    word_token.extend(words)
print(word_token[:3])

# 토큰화(문단 --> 단어) append: 2차원 데이터프레임으로
word_token2=[]
for sentence in sentences:
    words=nltk.word_tokenize(sentence)
    word_token2.append(words)
print(word_token2[:3])

In [ ]:
# 토큰화 함수: 문서의 모든 단어
from nltk import sent_tokenize, word_tokenize

def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentense) for sentense in sentences]
    return word_tokens
word_tokens = tokenize_text(text)
print(word_tokens[:2])

In [ ]:
# 1-3.불용어(stop_words) 제거
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

all_tokens=[]
for sentence in word_tokens:
    filtered_words=[]
    for word in sentence:
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
all_tokens[:1]

In [ ]:
# 1-4. 어근(형태소) 추출: Stemming/ Lemmatization -영어에서만 사용
from nltk import LancasterStemmer
stemmer=LancasterStemmer()
stemmer.stem('working')

In [ ]:
# 2.피처 벡터화/축출(BOW : CountVectorizer/TfidfVectorizer)
# 2-1. 카운트 기반의 벡터화: CountVectorizer
#전처리: 한글의 경우
import re
def cleaning(text):
    p = re.compile('[^ a-zA-Zㄱ-ㅣ가-힣]+')  #영어 ('[^ a-zA-Z0-9\.]+')
    result = p.sub('',text).lower()
    return result
df['content']=df['content'].apply(cleaning)

from konlpy.tag import Okt
def okt_tokenizer(text):   # 자르기 + 조사 삭제
    words=okt.pos(text, stem=True)
    filtered_words=[]
    for word, pos in words:
        if pos not in ['Josa']:
            filtered_words.append(word)
    return filtered_words

with open('data/stopword.txt','r',encoding='utf-8') as f:
    word = f.read()
    stopwords = word.split('\n')

# 셋트 만들기
y_df=review_df['sentiment']
X_df=review_df.drop(['id','sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test= train_test_split(X_df, y_df, test_size=0.3, random_state=156)
print(X_train.shape, X_test.shape)

# 벡터화
from sklearn.feature_extraction.text import CountVectorizer
cnt_vct=CountVectorizer(max_df=0.95,max_features=1000, min_df=2,
                        tokenizer=okt_tokenizer, stop_words=stopwords)
# 영어 count_vect = CountVectorizer(max_df=0.95,max_features=1000, min_df=2,stop_words='english', token_pattern = '[a-zA-Z]+',ngram_range=(1,2))
cnt_vct.fit(X_train)   # y값 없음!!
X_train_cnt=cnt_vct.transform(X_train)
X_test_cnt=cnt_vct.transform(X_test)

# 3.모델로 분류
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt, y_train)
pred=lr_clf.predict(X_test_cnt)
print('예측 정확도 : ', accuracy_score(y_test, pred))

In [ ]:
# 2-2. TF-IDF: TfidfVectorizer
# (자주 나오는 단어에 가중치/모든 문서에서 자주나오는 단어에 패털티)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vct=TfidfVectorizer(max_df=0.95,max_features=1000, min_df=2,
                          tokenizer=okt_tokenizer, stop_words=stopwords)
# 영어 count_vect = CountVectorizer(max_df=0.95,max_features=1000, min_df=2,stop_words='english', token_pattern = '[a-zA-Z]+',ngram_range=(1,2))
tf_vct.fit(X_train)
X_train_tf=tf_vct.transform(X_train)
X_test_tf=tf_vct.transform(X_test)

# 3.모델로 분류
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tf, y_train)
pred=lr_clf.predict(X_test_tf)
print('예측 정확도 accuracy: ', accuracy_score(y_test, pred))
print('예측 정확도 f1: ', f1_score(y_test,pred)
      
# 4. 적용
test_text = '욕나온다. 쓰레기'
predict = tf_vct.transform([test_text])
lr_clf.predict(predict)

In [ ]:
# 3-1. GridSearchCV로 파라미터 성능 향상
params={'C':[0.01, 0.1, 1, 5, 10]}
gr_lr_clf = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy', verbose=1)
gr_lr_clf.fit(X_train_tf , y_train)
print('LogisticRegression의 최적 파라미터: ', gr_lr_clf.best_params_)

# 최적 C값으로 예측, 정확도 평가
pred=gr_lr_clf.predict(X_test_tf)
print('예측 정확도 : ', accuracy_score(y_test, pred))

In [ ]:
# pipeline으로 만들기:TF-IDF 벡터화, GridSearchCV 최적찾기
from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tf_vct', TfidfVectorizer(stop_words='english')),
                   ('lr_clf', LogisticRegression())])
params={'tf_vct__ngram_range': [(1,1),(1,2)],
        'tf_vct__max_df': [100,200,300],
        'lr_clf__C': [1,5,7]}
grid_cv_pipe=GridSearchCV(pipeline, param_grid=params,cv=3,scoring='accuracy')
grid_cv_pipe.fit(X_train,y_train)
print('최적 파라미터: ', grid_cv_pipe.best_params_)
pred=gr_lr_clf.predict(X_test)
print('예측 정확도 : ', accuracy_score(y_test, pred))

In [ ]:
# 한글 단어 토큰
from konlpy.tag import Okt
okt = Okt()
words = []
for sentence in sentences:
    word = okt.morphs(sentence)
    words.append(word)
print(words[:2])
test_text = '나는 정말로 파이썬을 좋아한다. 아니 머신러닝을 더 좋아한다.'
print('normalize :', okt.normalize(test_text)) # 문장으로 추출
print('morphs :', okt.morphs(test_text))       # 구문 분석
print('nouns :', okt.nouns(test_text))         # 명사만
print('phrases :', okt.phrases(test_text))     # 구문
print('pos :', okt.pos(test_text))             # 품사와 함께 값고 함께 

#### 감성 분석: 지도학습, 비지도학습

In [2]:
# 영화 평가 분석// 지도학습 기반  (1) CountVectorizer + pipeline
review_df = pd.read_csv('data/labeledTrainData.tsv', header=0, sep="\t", quoting=3)

In [3]:
import re
review_df['review'] = review_df['review'].str.replace('<br />',' ')
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )

In [4]:
y_df=review_df['sentiment']
X_df=review_df.drop(['id','sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test= train_test_split(X_df, y_df, test_size=0.3, random_state=156)
print(X_train.shape, X_test.shape)

(17500, 1) (7500, 1)


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([('cnt_vect', CountVectorizer(stop_words='english',ngram_range=(1,2) )),
                     ('lr_clf', LogisticRegression(solver='liblinear', C=10))])

In [ ]:
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]
print(f'예측 정확도는 {accuracy_score(y_test ,pred):.4f}')
print(f'ROC-AUC는 {roc_auc_score(y_test, pred_probs):.4f}')

#### 영화 평가 분석// 지도학습 기반 (2) 한글

In [7]:
# 전처리(한글)
def cleaning(text):
    p = re.compile('[^ a-zA-Zㄱ-|가-힣\]+')
    result = p.sub('',text).lower()
    return result

from konlpy.tag import Okt
def okt_tokenizer(text):   # 자르기 + 조사 삭제
    words=okt.pos(text, stem=True)
    filtered_words=[]
    for word, pos in words:
        if pos not in ['Josa','KoreanParticle']:
            filtered_words.append(word)
    return filtered_words

with open('data/stopword.txt','r',encoding='utf-8') as f:
    word = f.read()
    stopwords = word.split('\n')

In [8]:
X_train, X_test, y_train, y_test=train_test_split(df['content'],df['score'], test_size=0.2, random_state=0 ) 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vct=TfidfVectorizer(tokenizer=okt_tokenizer, stop_words=stopwords, max_features=5000)
tfidf_vct.fit(X_train)
X_train_tf=tfidf_vct.transform(X_train)
X_test_tf=tfidf_vct.transform(X_test)

In [11]:
lr_clf=LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tf, y_train)
pred=lr_clf.predict(X_test_tf)
print('예측 정확도 : ', accuracy_score(y_test, pred))

예측 정확도 :  0.5818540433925049


#### LDA 토픽 모델링 (1): CountVectorizer만 허용

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 
    'comp.windows.x', 'talk.politics.mideast', 
    'soc.religion.christian', 'sci.electronics', 'sci.med' ]
news_df= fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), categories=cats, random_state=0)

In [ ]:
count_vect = CountVectorizer(max_df=0.95,max_features=1000, min_df=2,stop_words='english', token_pattern = '[a-zA-Z]+',ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)

lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)
print(lda.components_.shape)
print(lda.components_)  # 볼 수 없음, 숫자만 가득

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)
feature_names = count_vect.get_feature_names_out()
display_topics(lda, feature_names, 15)     # 가장 연관도 높은 word 15개 보기

#### LDA 토픽 모델링-(2) 한글

In [ ]:
df = pd.read_csv('data/petition.csv')
cats = ['정치개혁', '인권/성평등', '안전/환경', '교통/건축/국토', '육아/교육']
df_cats = df[df['category'].isin(cats)]
df_samples = df_cats.sample(frac=0.05, random_state = 0)
df_samples['category'].value_counts()

In [ ]:
import re
df_samples['content'] = df_samples['content'].apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', '', x))

from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem = True)
    return tokens_ko

with open('data/stopword.txt','r',encoding='utf-8') as f:
    word = f.read()
stopwords = word.split('\n')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_df = 0.9,max_features = 1000,min_df = 2, ngram_range = (1, 2),tokenizer = okt_tokenizer,stop_words = stopwords)
tfidf_df = tfidf_vect.fit_transform(df_samples['content'])

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5, random_state = 0)
lda.fit(tfidf_df)
print(lda.components_.shape)
print(lda.components_)

In [ ]:
feature_names = tfidf_vect.get_feature_names_out()
display_topics(lda, feature_names, 10)

In [ ]:
## LDA 토픽 모델링-(3) 한글/ 하나의 주제에서 주요 주제 뽑기

In [ ]:
df = pd.read_csv('data/petition.csv')

In [70]:
cats = ['육아/교육']
df_cats = df[df['category'].isin(cats)]
df_samples = df_cats.sample(frac=0.5, random_state = 0)
df_samples['content'] = df_samples['content'].apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', '', x))

In [ ]:
tfidf_vect = TfidfVectorizer(max_df = 0.9, max_features = 1000,min_df = 2, ngram_range = (1, 2),tokenizer = okt_tokenizer,stop_words = stopwords)
tfidf_df = tfidf_vect.fit_transform(df_samples['content'])   

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5, random_state = 0)    
lda.fit(tfidf_df)
feature_names = tfidf_vect.get_feature_names_out()
display_topics(lda, feature_names, 15)    # 토픽보기 함수 display_topics  15개

In [ ]:
lda.components_.argsort()[::-1][:5]

In [ ]:
lda_df=pd.DataFrame(lda.components_, columns=tfidf_vect.get_feature_names_out())
lda_df

#### KMeans 군집

In [ ]:
# 앞에 이미: 불러오기/ 클리닝/ TfidfVectorizer 벡터화,학습,변환
from sklearn.cluster import KMeans
km_cluster = KMeans(n_clusters = 4, max_iter = 10000, random_state = 0)
km_cluster.fit(tfidf_df)
cluster_label = km_cluster.labels_
clust_df = df_samples[['content', 'category']]
clust_df['cluster'] = cluster_label
clust_df.head(3)

In [ ]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명을 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    for cluster_num in range(clusters_num):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
    return cluster_details

In [ ]:
# 보기좋게 함수  get_cluster_details 를 표현하는 함수
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('# Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
clust_centers = km_cluster.cluster_centers_

# 위 두 함수 불러오기
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=clust_df, 
                                      feature_names=feature_names, clusters_num=4, top_n_features=10 )
print_cluster_details(cluster_details)

#### 유사도 분석 cosine_similarity

In [ ]:
# 텍스트 전처리(정규화): 클렌징/토큰화/불용어/형태소 축출

In [ ]:
tfidf_vect = TfidfVectorizer(max_df = 0.85, min_df = 2, tokenizer=okt_tokenizer, stop_words=stopwords, max_features=1000)       
feature_vect = tfidf_vect.fit_transform(df['content'])  

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_pair = cosine_similarity(feature_vect)
print(similarity_pair)
print('shape:',similarity_pair.shape)

In [ ]:
pd.DataFrame(feature_vect_simple.toarray(), columns=tfidf_vect_simple.get_feature_names_out())

#### 감정사전 함수(Rexicon)

In [ ]:
def setiment_analyzer(text):
    import pandas as pd
    from konlpy.tag import Kkma
    from nltk.util import ngrams
    
    senti_words = pd.read_csv('./data/polarity.csv')
    kkma = Kkma()
    ngram1 = kkma.pos(text, join = True)
    ngram2 = list(ngrams(ngram1, n=2)) # result를 ngram1로 바꿈
    new_ngram2 = []
    for n in ngram2:
        new_ngram2.append(';'.join(n))
    ngram3 = list(ngrams(ngram1, n=3))  # result를 ngram1로 바꿈
    new_ngram3 = []
    for n in ngram3:
        new_ngram3.append(';'.join(n))
    words = ngram1 + new_ngram2 + new_ngram3
    result_df = senti_words[senti_words['ngram'].isin(words)]

    neg_df = result_df[result_df['max.value'] == 'NEG']
    pos_df = result_df[result_df['max.value'] == 'POS']
    neg_value = (neg_df['NEG'] / neg_df['freq']).sum()
    pos_value = (pos_df['NEG'] / pos_df['freq']).sum()
    neg_length = neg_df.shape[0]
    pos_length = pos_df.shape[0]

    if pos_length == 0:
        final_value = (pos_value ) - (neg_value / neg_length)
    elif neg_length == 0:
        final_value = (pos_value / pos_length) - (neg_value)
    else:
        final_value = (pos_value / pos_length) - (neg_value / neg_length)


    if final_value >= 0:
        print('긍정문장입니다.')
    else:
        print('부정문장입니다.')
    return final_value

In [ ]:
text = df['content'].sample(1).iloc[0]
print(text)
setiment_analyzer(text)

In [ ]:
# 독립변수들이 원핫형식(희소행렬)일때 한 독립변수로 모으는 법

y = df.iloc[:, 1:]
y_label = pd.DataFrame({'target': y.columns})

from sklearn.preprocessing import OneHotEncoder
oh_enc = OneHotEncoder()
oh_enc.fit(y_label)

y_oh = y[oh_enc.categories_[0]]
y_oh.drop([5876, 11942], inplace=True)
oh_enc.inverse_transform(y_oh)

data = df[['문장']]
data.drop([5876, 11942], inplace=True)
data[['정답']] = oh_enc.inverse_transform(y_oh)

#### 추천시스템

In [ ]:
## 콘텐츠 기반 추천시스템

In [ ]:
movies =pd.read_csv('../data/tmdb_5000_movies.csv')
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

#데이터 가공
from ast import literal_eval         # literal_eval 리스트 등의 집합적 코드만 실행하여 안전을 기함/ eval은 모두 다 됨
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df[['genres','keywords']]
# name값 리스트객체로 만들기
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])  # list로 가져오기
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])  # list로 가져오기

# 내용이 많으니 단어들을 한데 묶어서 벡터화 시킴(공백으로 word단위가 구분되는 문자열로 반환
from sklearn.feature_extraction.text import CountVectorizer
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
# 유사도가 큰 순서로 정렬후 인덱스 반환
genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]  # 무조건 1개 밖에 추천 못함// [::-1]은 내림차순 [::1]은 오름차순

# 장르 유사도에 따라 영화를 추천하는 함수
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    # top_n index는 2차원이므로 데이터프레임 index로 사용하기위해서 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]
    
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',5)
similar_movies

In [ ]:
#장르만으로 추천할때는 모두 같은 장르만 나오므로 추천시스템 문제가 있음 
np.sort(genre_sim[2731])[::-1][:10] 
# 같은 장르 + 평점 높은 작품 먼저 추천 - 이것도 투표수 적을땐 객관성 없음
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]
## 장르 유사도에 따른 영화 추천
C = movies_df['vote_average'].mean()       # 전체 영화에 대한 평균평점
m = movies_df['vote_count'].quantile(0.5)  # 평점을 부여하기 위한 최소횟수
print('C:',round(C,3), ', m:',round(m,3))

In [ ]:
# 장르 유사도 함수(가중치 계산)
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile) # 평점을 부여하기 위한 최소횟수
C = movies_df['vote_average'].mean()             # 전체 영화에 대한 평균평점
def weighted_vote_average(record):
    v = record['vote_count']                     # 개별 영화에 평점을 투표한 수
    R = record['vote_average']                   # 개별 영화에 대한 평균 평점
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C ) 
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)   # 칼럼 추가
movies_df['weighted_vote'] 

# 가중치 변수로 추천영화 정렬
movies_df[['title','vote_average','weighted_vote']].sort_values('weighted_vote', ascending = False)[:10]

# 투표수로 정렬/ 투표수를 감안하지 않아서 점수는 높지만 문제 있음
movies_df[['title','vote_average','weighted_vote']].sort_values('vote_average', ascending = False)[:10]

In [ ]:
## 아이템 기반 최근접이웃 협업 필터링

In [ ]:
# 데이터 가공: NaN값 제거 등
# 영화간 유사도 산출
# 영화(아이템) 간 유사도 산출  : 평점만 가지고 유사도 산출
from sklearn.metrics.pairwise import cosine_similarity
ratings_matrix_T = ratings_matrix.transpose()
item_sim = cosine_similarity(ratings_matrix_T)
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
# 가장 가까운(유사도) 영화 뽑아보기
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

# 개인 예측 평점 함수(가중치 계산)
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)]) # 계산을 위해 행렬로 바꿈
    return ratings_pred
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)

# 유저별 평점 예측 - 빈칸채우기 --> 실제 유저가 평가한 점수들과도 오차가 큼/ 그래도 의미가 있음
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index, columns = ratings_matrix.columns)

# 평가해보기
from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):                         # actual 실제데이터/ 비어있는 상태
    pred = pred[actual.nonzero()].flatten()        # 그래서 제로가 아닌 것들만 가져옴   
    actual = actual[actual.nonzero()].flatten()     
    return np.sqrt(mean_squared_error(pred, actual))  # 이미 루트값이므로 
print('아이템 기반 모든 인접 이웃 RMSE: ', get_mse(ratings_pred, ratings_matrix.values ))

# 예측 평점 계산, 실제 평점과의 MSE 구하기
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):  # top 20개만    
    pred = np.zeros(ratings_arr.shape)            # 유저-아이템 행렬만큼 0행렬 생성 
    for col in range(ratings_arr.shape[1]):       # 아이템 개수 만큼 수행
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]  
        for row in range(ratings_arr.shape[0]):       # 모든 유저에 대한
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))  # /= 왼쪽 변수에서 오른쪽 값을 나누고 결과를 왼쪽변수에 할당
    return pred
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=10)
print('아이템 기반 인접 TOP-10 이웃 RMSE: ', get_mse(ratings_pred, ratings_matrix.values ))


In [ ]:
# 행렬 분해를 이용한 잠재요인 협업 필터링 실습

In [ ]:
## 잠재요인 협업 필터  : 차원축소의 잠재요인 (가중치), 경사하강법) ==> 모델기반 협업(알고리즘적 예측)
# 잠재요인 협업필더링 실습 : 오차계산
def get_rmse(R, P, Q, non_zeros): # P, Q 예측행렬/ R 실제값
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱(np.dot)으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T) 
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
# 확률적 경사하강법 학습 함수/ 행렬분해 예제
def matrix_factorization(R, K, steps=100, learning_rate=0.01, r_lambda = 0.01):
    # 만약 데이터프레임이 들어오면
    if type(R) == 'pandas.core.frame.DataFrame':
        R = R.values

    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./ K, size=(num_users, K)) 
    Q = np.random.normal(scale=1./ K, size=(num_items, K))  # 나중에 전치시킬 예정
    break_count = 0
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]  # 빈칸에 0넣기/ 0아닌 곳은 남기기
    # 이전 코드에 이어서 작성
    # SGD기법(차원축소)으로 P와 Q 매트릭스를 계속 업데이트.   
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값(r)과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])  # P[i,:] 가중치  *** 중요
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])  # Q[j,:]  가중치  ** 중요 
            # // 둘다 가동하면 속도문제, 결과 문제있음: 번갈아가면서 실행하는 ASL기법(SGD와 비교) 있음
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
    return P, Q

In [ ]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=100, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

# 예측 평점 출력
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,columns = ratings_matrix.columns)


In [ ]:
# 영화 추천
# 9번 사용자가 관람하지 않는 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)
# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)
# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies